In [61]:
#import re
import sys, codecs, optparse, glob, os
import string, yaml
import random, warnings
from ToolsMoodleXML2 import *

In [62]:
nKalaba="K4"
dossier="/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-%s"%nKalaba
enonces=["Theme","Version"]
enonces=["Version"]
theme="Traductions"
enonces=["Ordre"]
theme="Syntagmes"

In [63]:
from os.path import expanduser
home = expanduser("~")
repertoire=home+dossier
base=repertoire+"/"

In [64]:
def chaine2utf8(chaine):
    if type(chaine)==str:
        result=unicode(chaine.decode('utf8'))
    elif type(chaine)==unicode:
        result=chaine
    return result

In [65]:
def substituerChamps(chaine, substituts):
    result=chaine
    for substitut in substituts:
        result=result.replace(substitut,substituts[substitut])
    return result

In [66]:
def preparationDonnees():
    for ligne in donneesLignes:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            temp=ligne.strip().split(";")
            result={"boucle":[temp],"conclusion":[]}
            titre=makeChamps(configuration[u"Titre"],temp).encode('utf8')
            donnees.append({titre:result})
    return

In [67]:
def numerotationDonnees(donnees):
    result={}
    for nDonnee, donnee in enumerate(donnees):
        result[u"#%d#"%(nDonnee+1)]=chaine2utf8(donnee)
    return result

In [68]:
def preparationConsigne(consigne,clozes,substituts):
    result=[]
    reponse=[]
    for element in consigne:
        chaine=chaine2utf8(element)
        if chaine in clozes:
            champ=substituerChamps(chaine,substituts)
            if clozes[chaine]=="TXT":
                result.append(champ)
            else:
                reponse.append(champ)
        else:
            result.append(chaine)
    return (result,reponse)

In [69]:
for enonce in enonces:
    yamlFichier=base+theme+"-"+enonce+".yaml"
    csvFichier=base+theme+".txt"
    boolEX=False
            
    with open(yamlFichier, 'r') as stream:
        configuration=yaml.load(stream)

    with open(csvFichier, 'r') as file:
        donneesLignes=file.readlines()

    donnees=[]

    preparationDonnees()

        
    categorie=configuration[u"Catégorie"]+"/RX"
    consigne=[configuration[u"Boucle"][u"début"]]+configuration[u"Boucle"][u"contenu"]
    boucleCLOZE=configuration[u"Boucle"][u"Cloze"]
    globalWrap=[configuration[u"Décor"][u"fond"]+configuration[u"Décor"][u"entete"]+configuration[u"Global"][u"début"],configuration[u"Global"]["fin"]]
    questions=XMLSeries(categorie)
    clozes=numerotationDonnees(boucleCLOZE)
    for exercices in donnees:
        for exercice in exercices:
            titre=chaine2utf8(exercice)
            substituts=numerotationDonnees(exercices[exercice]['boucle'][0])
            regexConsigne,regexReponse= preparationConsigne(consigne,clozes,substituts)
            regexConsigne=regexConsigne
            questions.addExercice(RegExExercice(titre,[globalWrap[0]]+regexConsigne,regexReponse,usehint="2"))

    xmlOutput=MoodleXML()
    xmlOutput.addQuiz(questions.getSeries())

    xmlFichier=yamlFichier.split(".")[0]+"-RX.xml"

    with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
        output.write(xmlOutput.getXML())


In [70]:
#print xmlFichier

In [71]:
#questionsConsigne.getConsigne(donnees)

In [72]:
#print xmlOutput.getXML()